In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#from sklearn.multioutput import MultiOutputRegressor
from src.algorithms.multioutput import MultiOutputRegressor, RegressorChain

from sklearn.utils import shuffle
#from catboost import CatBoostRegressor
from src.algorithms.catboost_model import CatBoost

from sklearn.metrics import mean_absolute_error

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

import src.data as data
import src.utility as utility

In [3]:
#dataset_train = base_df.merge(data.base_dataset(), how='left')
X, Y = data.dataset_with_features('train', onehot=False, drop_index_columns=True)

caching merged dataset train


In [4]:
print(X.shape)
print(Y.shape)
print()
print(Y.dtypes)

(33202, 77)
(33202, 4)

SPEED_AVG_Y_0    float64
SPEED_AVG_Y_1    float64
SPEED_AVG_Y_2    float64
SPEED_AVG_Y_3    float64
dtype: object


## Preprocess data

In [4]:
X.head(10)

,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,TEMPERATURE_-4,TEMPERATURE_-3,TEMPERATURE_-2,TEMPERATURE_-1,MIN_TEMPERATURE_-4,MIN_TEMPERATURE_-3,MIN_TEMPERATURE_-2,MIN_TEMPERATURE_-1,MAX_TEMPERATURE_-4,MAX_TEMPERATURE_-3,MAX_TEMPERATURE_-2,MAX_TEMPERATURE_-1
0,1,2,1,11,Veicolo_in_avaria,NaN,199.6642,199.3150,204.7759,NaN,41.4457,52.0643,43.0659,NaN,263.734,292.858,294.476,NaN,59.866,59.866,59.866,NaN,107.0,129.0,139.0,NaN,Quasi Sereno,Quasi Sereno,Quasi Sereno,NaN,38.0,38.0,38.0,NaN,28.0,28.0,28.0,NaN,21.0,21.0,21.0,NaN,29.0,29.0,29.0
1,1,2,1,6,Gestione_viabilita,NaN,NaN,NaN,208.9124,NaN,NaN,NaN,38.8807,NaN,NaN,NaN,352.724,NaN,NaN,NaN,59.866,NaN,NaN,NaN,238.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2,1,6,Gestione_viabilita,NaN,NaN,208.9124,201.9654,NaN,NaN,38.8807,39.7029,NaN,NaN,352.724,286.386,NaN,NaN,59.866,59.866,NaN,NaN,238.0,216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,1,6,Gestione_viabilita,NaN,NaN,NaN,213.5957,NaN,NaN,NaN,35.7280,NaN,NaN,NaN,334.926,NaN,NaN,NaN,59.866,NaN,NaN,NaN,246.0,NaN,NaN,NaN,Rovescio Nelle Vicinanze con Pioggia,NaN,NaN,NaN,38.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,19.0,NaN,NaN,NaN,21.0
4,1,2,1,2,Manutenzione_opere_in_verde,210.7883,213.6824,223.9108,223.1689,34.5211,33.1529,32.9331,30.7040,281.532,343.016,389.938,341.398,59.866,61.484,64.720,59.866,231.0,228.0,222.0,239.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,226.0,226.0,38.0,38.0,21.0,21.0,19.0,19.0,10.0,10.0,12.0,12.0,21.0,21.0,19.0,19.0
5,1,2,1,2,Manutenzione_opere_in_verde,223.1689,218.5277,219.2802,214.6672,30.7040,29.5927,28.9895,32.1558,341.398,328.454,300.948,286.386,59.866,59.866,61.484,59.866,239.0,265.0,255.0,258.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,38.0,38.0,38.0,38.0,19.0,19.0,19.0,19.0,12.0,12.0,12.0,12.0,19.0,19.0,19.0,19.0
6,1,2,1,2,Manutenzione_opere_in_verde,218.5277,219.2802,214.6672,216.8678,29.5927,28.9895,32.1558,29.6041,328.454,300.948,286.386,310.656,59.866,61.484,59.866,59.866,265.0,255.0,258.0,232.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,38.0,38.0,38.0,38.0,19.0,19.0,19.0,19.0,12.0,12.0,12.0,12.0,19.0,19.0,19.0,19.0
7,1,2,1,11,Veicolo_in_avaria,NaN,NaN,199.6642,199.3150,NaN,NaN,41.4457,52.0643,NaN,NaN,263.734,292.858,NaN,NaN,59.866,59.866,NaN,NaN,107.0,129.0,NaN,NaN,Quasi Sereno,Quasi Sereno,NaN,NaN,38.0,38.0,NaN,NaN,28.0,28.0,NaN,NaN,21.0,21.0,NaN,NaN,29.0,29.0
8,1,2,1,2,Manutenzione_opere_in_verde,NaN,216.4407,208.1131,217.6035,NaN,44.8391,42.4317,37.3565,NaN,368.904,281.532,349.488,NaN,59.866,59.866,59.866,NaN,183.0,186.0,231.0,NaN,Quasi Sereno,Quasi Sereno,Quasi Sereno,NaN,38.0,38.0,38.0,NaN,12.0,12.0,14.0,NaN,13.0,13.0,12.0,NaN,14.0,14.0,14.0
9,1,2,1,2,Manutenzione_opere_in_verde,211.3166,212.3555,207.6177,213.4945,27.7924,37.2444,34.9689,33.5346,278.296,307.420,273.442,296.094,61.484,59.866,59.866,59.866,280.0,232.0,274.0,258.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,38.0,38.0,226.0,226.0,15.0,15.0,17.0,17.0,12.0,12.0,10.0,10.0,14.0,14.0,15.0,15.0


In [7]:
# add features
import src.preprocessing.other_features as feat
avg_speed_road_event = feat.avg_speed_for_roadtype_event()
X = X.merge(avg_speed_road_event, how='left', on=['EVENT_TYPE','ROAD_TYPE'])

avg_speed_road_event.head()

In [5]:
X.fillna(0, inplace=True)
#Y.fillna(0, inplace=True)

weather_cols = [f'WEATHER_{i}' for i in range(-10,0)]
categorical_cols = ['EMERGENCY_LANE', 'ROAD_TYPE', 'EVENT_DETAIL','EVENT_TYPE',] + weather_cols

## Train/Validation split

In [10]:
#X, Y = shuffle(X, Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [10]:
def evaluate(model, X_test, y_test):
    mask_test = np.all(y_test.notnull(), axis=1)
    print('number of valid samples:', (mask_test*1).sum())

    y_pred = model.predict(X_test[mask_test])
    return mean_absolute_error(y_test[mask_test], y_pred)

## Fit

In [6]:
catboost = CatBoost(params_dict={
    'X': X,
    'loss_function': 'MAE',
    'eval_metric': 'MAE',
    'iterations':2,
    'depth':6,
    'learning_rate':1,
    'cat_features': categorical_cols,
    'verbose': False
})

In [7]:
model = MultiOutputRegressor(catboost, n_jobs=2)
model.fit(X, Y)

KeyboardInterrupt: 

In [8]:
model = RegressorChain(catboost)
model.fit(X, Y)

catboost: fitting...
catboost: fitting...
catboost: fitting...
catboost: fitting...


RegressorChain(base_estimator=<src.algorithms.catboost_model.CatBoost object at 0x1a20daf3c8>,
        cv=None, order=None, random_state=None)

In [11]:
evaluate(X_test, y_test)

7.8075137571574285

## Baseline

Catboost chain (no features + weather): 7.076924911258332 TEST: 7.9718490219958404

Catboost chain: 7.8075137571574285

Catboost chain (avg speed x road type): 7.096708023412791

Catboost chain (avg speed x road type + avg speed sensor + avg_speed_road): 6.79054199721705

## Test model

In [6]:
import src.algorithms.inout as inout

In [7]:
X, Y = data.dataset_with_features('test', onehot=False, drop_index_columns=True)
X.fillna(0, inplace=True)
X.shape

caching merged dataset test


(7055, 45)

In [6]:
X.head()

,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-10,SPEED_AVG_-9,SPEED_AVG_-8,SPEED_AVG_-7,SPEED_AVG_-6,SPEED_AVG_-5,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-10,SPEED_SD_-9,SPEED_SD_-8,SPEED_SD_-7,SPEED_SD_-6,SPEED_SD_-5,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-10,SPEED_MAX_-9,SPEED_MAX_-8,SPEED_MAX_-7,SPEED_MAX_-6,SPEED_MAX_-5,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-10,SPEED_MIN_-9,SPEED_MIN_-8,SPEED_MIN_-7,SPEED_MIN_-6,SPEED_MIN_-5,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-10,N_VEHICLES_-9,N_VEHICLES_-8,N_VEHICLES_-7,N_VEHICLES_-6,N_VEHICLES_-5,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-10,WEATHER_-9,WEATHER_-8,WEATHER_-7,WEATHER_-6,WEATHER_-5,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-10,DISTANCE_-9,DISTANCE_-8,DISTANCE_-7,DISTANCE_-6,DISTANCE_-5,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,avg_speed_street,avg_speed_sensor
0,1.0,2.0,1.0,6.0,Gestione_viabilita,NaN,NaN,NaN,NaN,190.1505,187.4472,189.4691,205.3692,205.0545,212.1632,NaN,NaN,NaN,NaN,39.6333,39.4972,32.4724,45.5916,40.9371,33.4045,NaN,NaN,NaN,NaN,349.488,266.970,279.914,323.600,313.892,318.746,NaN,NaN,NaN,NaN,59.866,59.866,59.866,59.866,59.866,59.866,NaN,NaN,NaN,NaN,205.0,215.0,258.0,194.0,225.0,205.0,NaN,NaN,NaN,NaN,Quasi Sereno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,226.0,NaN,NaN,NaN,NaN,NaN,219.711349,216.926568
1,1.0,2.0,1.0,4.0,Ostacolo_in_carreggiata,NaN,NaN,NaN,NaN,NaN,215.4850,222.5566,222.8882,222.6499,225.3653,NaN,NaN,NaN,NaN,NaN,30.5128,30.8295,32.4691,32.9367,36.0934,NaN,NaN,NaN,NaN,NaN,297.712,288.004,349.488,357.578,336.544,NaN,NaN,NaN,NaN,NaN,71.192,59.866,63.102,59.866,59.866,NaN,NaN,NaN,NaN,NaN,228.0,238.0,233.0,222.0,220.0,NaN,NaN,NaN,NaN,NaN,NaN,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,NaN,NaN,NaN,NaN,NaN,NaN,90.0,90.0,90.0,90.0,219.711349,216.926568
2,1.0,2.0,1.0,4.0,Ostacolo_in_carreggiata,NaN,NaN,NaN,NaN,215.4850,222.5566,222.8882,222.6499,225.3653,218.1919,NaN,NaN,NaN,NaN,30.5128,30.8295,32.4691,32.9367,36.0934,35.7296,NaN,NaN,NaN,NaN,297.712,288.004,349.488,357.578,336.544,347.870,NaN,NaN,NaN,NaN,71.192,59.866,63.102,59.866,59.866,59.866,NaN,NaN,NaN,NaN,228.0,238.0,233.0,222.0,220.0,231.0,NaN,NaN,NaN,NaN,NaN,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,NaN,NaN,NaN,NaN,NaN,90.0,90.0,90.0,90.0,90.0,219.711349,216.926568
3,1.0,2.0,1.0,6.0,Gestione_viabilita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190.1505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.6333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,349.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quasi Sereno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,226.0,219.711349,216.926568
4,1.0,2.0,1.0,4.0,Ostacolo_in_carreggiata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.4166,225.7645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.3690,35.3666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330.072,333.308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.956,61.484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,220.0,227.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quasi Sereno,Quasi Sereno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,226.0,226.0,219.711349,216.926568


In [17]:
import src.preprocessing.other_features as feat
avg_speed_road_event = feat.avg_speed_for_roadtype_event()
X = X.merge(avg_speed_road_event, how='left', on=['EVENT_TYPE','ROAD_TYPE'])

caching resources/dataset/originals/speeds_train.csv.gz
caching resources/dataset/preprocessed/events_train.csv.gz


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3291: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



caching sensors

sensors loaded in: 0.014 s

shape of the dataframe is: (2171, 5)


In [8]:
#model = inout.load('saved_models/catboost_chain_6.79054.jl')
model = inout.load('saved_models/catboost_chain_7.07692_nofeatures.jl')

In [11]:
evaluate(model, X_test=X, y_test=Y)

number of valid samples: 6106


7.9718490219958404